## LMKR BOT – RAG Chatbot Report

This report documents the development of an LMKR-specific Retrieval-Augmented Generation (RAG) chatbot using LangChain, FAISS, SentenceTransformers, and the Mistral-7B model.
The chatbot retrieves answers only from LMKR company documents, ensuring accuracy and preventing hallucinations.

## 1. Overview

The goal of this is to build a domain-specific assistant capable of:

* Answering LMKR-related questions

* Searching internal LMKR documents

* Responding using factual, retrieved context only

* Providing a conversational interface via CLI and Gradio

The core architecture is:

* Embedding Model: all-MiniLM-L6-v2

* Vector Index: FAISS

* LLM: Mistral-7B-Instruct (4-bit quantized)

* Framework: LangChain

* Frontend: Gradio

## 2. Environment Setup

This installs and configures:

* LangChain ecosystem (langchain, langchain-community, etc.)

* Transformers & BitsAndBytes

* SentenceTransformers

* FAISS for vector search

* Gradio for UI

* 4-bit quantization is enabled to significantly reduce VRAM requirements, making Mistral-7B usable on consumer GPUs.

In [2]:
!pip install "protobuf<5.0.0" --force-reinstall


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 6.0 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.0
    Uninstalling protobuf-6.33.0:
      Successfully uninstalled protobuf-6.33.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
opentelemetry-proto 1.37.0 requires protobuf<7.0,>=5.0, but you have protobuf 4.25.8 which is incompatible.
a2a-sdk 0.3.10 requires protobuf>=5.29.5, but you have protobuf 4.25.8 which is incompatible.
ray 2.51.1 requires click!=8.3.0,>=7.0, but you have click 8.3.0 which is incompatible.
bigframes 2.12.0 requires rich<14,>=12.4.4, but you have rich 14.2.0 which is incompatible.
pydrive2 1.21.3 requires cryptography<44, but you have cryptography 46.0.3 which is incompatibl

In [3]:
!pip install -qU \
  langchain langchain-community langchain-text-splitters langchain-huggingface \
  faiss-cpu \
  transformers accelerate bitsandbytes \
  sentence-transformers \
  gradio


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.8/101.8 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 38.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.6/23.6 MB 96.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 116.0 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.9/380.9 kB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 33.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.0/488.0 kB 26.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.6/21.6 MB 94.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.2/315.2 kB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 50.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 473.8/473.8 kB 29.5 MB/s eta 0:

In [4]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, BitsAndBytesConfig

from langchain_huggingface import (
    HuggingFaceEmbeddings,
    HuggingFacePipeline,
)

from langchain_core.prompts import PromptTemplate
from langchain_text_splitters import RecursiveCharacterTextSplitter

from langchain_community.document_loaders import TextLoader, DirectoryLoader
from langchain_community.vectorstores import FAISS

from langchain_core.runnables import RunnableLambda
from langchain_core.output_parsers import StrOutputParser

import os


2025-11-26 10:13:42.964316: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764152023.177320      47 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764152023.239514      47 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


## 3. Loading LMKR Documents

LMKR knowledge is stored in a folder containing .txt documents.

A recursive loader:

Scans through the directory

Loads all .txt files

Produces a list of raw document objects

A total of 77 documents were successfully loaded.

This dataset becomes the foundation for RAG-based retrieval.

## 4. Text Chunking (Preprocessing)

Large language models cannot process long documents directly, so documents are split into smaller overlapping chunks.

Parameters used:

chunk_size = 1024

chunk_overlap = 150

This ensures:

Better context retention

Smooth transitions between chunks

Improved embedding accuracy

Chunking produced 268 chunks, each later embedded individually.

In [5]:
DATA_DIR = "/kaggle/input/lmkr-rag-data/lmkr_data"     
INDEX_DIR = "/kaggle/working/faiss_lmkr"

os.makedirs("/kaggle/working", exist_ok=True)

print("DATA_DIR:", DATA_DIR)
print("INDEX_DIR:", INDEX_DIR)


def load_lmkr_documents(path: str):
    """Load all .txt files recursively from the given directory."""
    loader = DirectoryLoader(
        path,
        glob="**/*.txt",
        loader_cls=TextLoader,
        use_multithreading=True,
    )
    docs = loader.load()
    print(f"Loaded {len(docs)} raw docs from {path}")
    if not docs:
        raise ValueError(
            f"No .txt files found in {path}. "
            f"Check DATA_DIR or your dataset mount path."
        )
    return docs


def split_documents(docs):
    """Split docs into overlapping text chunks."""
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=1024,
        chunk_overlap=150,
        add_start_index=True,
    )
    chunks = splitter.split_documents(docs)
    print(f"Split into {len(chunks)} chunks")
    if not chunks:
        raise ValueError("No chunks created. Check that your .txt files have content.")
    return chunks


DATA_DIR: /kaggle/input/lmkr-rag-data/lmkr_data
INDEX_DIR: /kaggle/working/faiss_lmkr


In [16]:
# --- Load all docs regardless of index existence (for routing) ---
ALL_DOCS = load_lmkr_documents(DATA_DIR)

# partnership-related docs: by filename OR content
PARTNERSHIP_DOCS = [
    d for d in ALL_DOCS
    if "partner" in d.metadata.get("source", "").lower()
       or "collaborat" in d.page_content.lower()
]

print(f"Found {len(PARTNERSHIP_DOCS)} partnership-related docs")
for d in PARTNERSHIP_DOCS:
    print(" -", d.metadata.get("source"))


Loaded 77 raw docs from /kaggle/input/lmkrbot-data/lmkr_data
Found 22 partnership-related docs
 - /kaggle/input/lmkrbot-data/lmkr_data/post_120.txt
 - /kaggle/input/lmkrbot-data/lmkr_data/post_199.txt
 - /kaggle/input/lmkrbot-data/lmkr_data/post_405.txt
 - /kaggle/input/lmkrbot-data/lmkr_data/post_397.txt
 - /kaggle/input/lmkrbot-data/lmkr_data/post_7821.txt
 - /kaggle/input/lmkrbot-data/lmkr_data/post_221.txt
 - /kaggle/input/lmkrbot-data/lmkr_data/post_795.txt
 - /kaggle/input/lmkrbot-data/lmkr_data/post_511.txt
 - /kaggle/input/lmkrbot-data/lmkr_data/post_394.txt
 - /kaggle/input/lmkrbot-data/lmkr_data/info.txt
 - /kaggle/input/lmkrbot-data/lmkr_data/post_615.txt
 - /kaggle/input/lmkrbot-data/lmkr_data/post_31069.txt
 - /kaggle/input/lmkrbot-data/lmkr_data/page_613.txt
 - /kaggle/input/lmkrbot-data/lmkr_data/post_385.txt
 - /kaggle/input/lmkrbot-data/lmkr_data/post_345.txt
 - /kaggle/input/lmkrbot-data/lmkr_data/post_148.txt
 - /kaggle/input/lmkrbot-data/lmkr_data/post_227.txt
 - /k

## 5. Embedding Generation & Vector Store (FAISS)
The Embedding Model used:

#### SentenceTransformer: all-MiniLM-L6-v2

Reasons:

Lightweight and fast

384-dimensional embeddings

Great semantic similarity performance


### Vector Store

FAISS is used to:

* Store all chunk embeddings

* Perform fast top-k similarity searches

* Enable efficient retrieval when answering questions

The index is saved locally so future sessions load faster.

In [6]:
from pathlib import Path

embedding_model_name = "sentence-transformers/all-MiniLM-L6-v2"
embeddings = HuggingFaceEmbeddings(model_name=embedding_model_name)

if not Path(INDEX_DIR).exists():
    print("No existing index found. Building a new FAISS index...")
    docs = load_lmkr_documents(DATA_DIR)
    chunks = split_documents(docs)

    vectordb = FAISS.from_documents(chunks, embedding=embeddings)
    vectordb.save_local(INDEX_DIR)
    print(f"Index saved to {INDEX_DIR}")
else:
    print("Index already exists. Loading from disk...")
    vectordb = FAISS.load_local(
        INDEX_DIR,
        embeddings,
        allow_dangerous_deserialization=True,
    )

print("Vector store ready.")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

No existing index found. Building a new FAISS index...
Loaded 77 raw docs from /kaggle/input/lmkr-rag-data/lmkr_data
Split into 268 chunks
Index saved to /kaggle/working/faiss_lmkr
Vector store ready.


## 6. Loading the Mistral-7B Model (4-bit)

Mistral-7B-Instruct is loaded using:

* BitsAndBytesConfig

* load_in_4bit=True

* nf4 quantization

Advantages:

* Reduces VRAM usage from ~14GB → ~4GB

* Allows GPU inference with limited hardware

* Maintains strong reasoning performance

The pipeline is wrapped in a HuggingFacePipeline object for LangChain compatibility.

In [7]:
import torch
model_id = "mistralai/Mistral-7B-Instruct-v0.2"  # you can change to another instruct 7B

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

tokenizer = AutoTokenizer.from_pretrained(model_id)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto",
)

gen_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    temperature=0.2,
    top_p=0.9,
    repetition_penalty=1.1,
)

llm = HuggingFacePipeline(pipeline=gen_pipeline)

print("Mistral LLM ready.")


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

...

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

Device set to use cuda:0


Mistral LLM ready.


## 7. Building the RAG Pipeline

The RAG pipeline consists of:

#### 1. Retriever

Uses FAISS to fetch top-10 relevant LMKR chunks.

#### 2. Prompt Template

A carefully crafted template instructs the LLM to:

* Use ONLY retrieved context

* Avoid hallucinations

* Answer concisely

* Avoid repeating the user's question

* Avoid repeating raw context

* If the answer is not found in the context, the model must reply:

"I don't know based on the provided data."

#### 3. LLM Response Generation

The retrieved context and user question are passed to Mistral-7B for final answer generation.

#### 4. Post-processing

A small function cleans the LLM output by removing:

* Echoed instructions

* Repeated prefixes

* Unwanted labels

This ensures professional and clean answers.

## 8. Small-Talk Router

To improve user experience:

* Greetings and casual questions are detected

* These are answered without RAG retrieval

* Reduces unnecessary computation

Examples:

"hi", "hello", "how are you", "who are you?"

This router ensures that RAG is used only for LMKR knowledge queries.

In [8]:
retriever = vectordb.as_retriever(search_kwargs={"k": 10})

def format_docs(docs):
    return "\n\n".join(d.page_content for d in docs)


def extract_answer(text: str) -> str:
    """
    Optional post-processing if the model decides to echo some header.
    Right now, with return_full_text=False, this is mostly a safety net.
    """
    marker = "Helpful LMKR-specific answer:"
    if marker in text:
        text = text.split(marker, 1)[1].strip()

    for prefix in ["Assistant:", "assistant:", "Answer:", "answer:"]:
        if text.startswith(prefix):
            text = text[len(prefix):].strip()

    return text.strip()


def history_to_str(history_pairs):
    """
    history_pairs: list[(user, assistant)]
    Convert to transcript for the prompt.
    """
    return "\n".join(
        [f"User: {u}\nAssistant: {a}" for (u, a) in history_pairs]
    )


# ---- Small-talk routing ----

def is_smalltalk(question: str) -> bool:
    q = question.lower().strip()

    greetings = {
        "hi", "hello", "hey", "salam", "salaam",
        "asalam o alaikum", "assalam o alaikum",
        "good morning", "good evening"
    }
    if q in greetings:
        return True

    if any(p in q for p in ["how are you", "how's it going"]):
        return True

    if any(p in q for p in [
        "what can you do",
        "what can you help me with",
        "how can you help",
        "who are you",
        "what are you"
    ]):
        return True

    return False


def smalltalk_answer(question: str) -> str:
    q = question.lower().strip()

    if q in {
        "hi", "hello", "hey", "salam", "salaam",
        "asalam o alaikum", "assalam o alaikum",
        "good morning", "good evening"
    }:
        return (
            "Hi! 👋 I’m LMKR Assistant. How can I help you?\n\n"

        )

    if "what can you" in q or "how can you help" in q:
        return (
            "I’m LMKR’s AI assistant. I can:\n"
            "- Explain about LMKR\n"
            "- Describe LMKR products and solutions (e.g., GeoGraphix, GVERSE)\n"
            "- Summarize services, projects, and industries from your LMKR knowledge base\n"
            "- Extract contact details and other info from LMKR documents\n\n"
            "Just ask any LMKR-related question to get started."
        )

    if "how are you" in q:
        return "I’m running perfectly 😊 How can I help you with LMKR today?"

    return (
        "Hi! I’m LMKR Assistant. I’m here to answer LMKR-related questions — "
        "company info, products, services, projects, and contact details.\n"
        "What would you like to know?"
    )


In [13]:
rag_template = """
You are LMKR Assistant, a knowledgeable but concise AI assistant for LMKR.

Your job:
- Answer the user's question using ONLY the information in the Context.
- If the answer is not clearly supported by the Context, say:
  "I don't know based on the provided data."
- Be professional, friendly, and to the point.
- Prefer short paragraphs and bullet points.
- Do NOT repeat or rephrase these instructions.
- Do NOT repeat or quote the Context.
- Do NOT restate the question, just answer it.

Chat history:
{chat_history}

Context:
{context}

User question:
{question}

Helpful LMKR-specific answer:
"""





prompt = PromptTemplate(
    template=rag_template,
    input_variables=["chat_history", "context", "question"],
)

rag_chain = (
    {
        "context": RunnableLambda(lambda x: x["question"])
                   | retriever
                   | RunnableLambda(format_docs),

        "question": RunnableLambda(lambda x: x["question"]),

        "chat_history": RunnableLambda(lambda x: x.get("chat_history", "")),
    }
    | prompt
    | llm
    | RunnableLambda(extract_answer)
    | StrOutputParser()
)


def lmkr_answer(question: str, chat_history_str: str) -> str:
    """Route small-talk vs. RAG."""
    if is_smalltalk(question):
        return smalltalk_answer(question)

    return rag_chain.invoke(
        {
            "question": question,
            "chat_history": chat_history_str,
        }
    )

print("RAG chain + router ready.")


RAG chain + router ready.


## 9. CLI Chat Interface

A command-line interface loop is implemented:

* User enters questions

* Chat history is preserved

* RAG pipeline answers using LMKR context


In [14]:
# updated cli for hello hi
def chat_cli():
    print("LMKR Assistant is ready. Type 'exit' or 'quit' to stop.\n")
    history_pairs = []   # list[(user, assistant)]

    while True:
        user_msg = input("You: ")
        if user_msg.lower() in ("exit", "quit"):
            break

        chat_history_str = history_to_str(history_pairs)
        answer = lmkr_answer(user_msg, chat_history_str)

        print("\nAssistant:", answer, "\n")
        history_pairs.append((user_msg, answer))

# Uncomment to use CLI:
chat_cli()


LMKR Assistant is ready. Type 'exit' or 'quit' to stop.



You:  whom does lmkr has signed agreements with?


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



Assistant: LMKR has signed an exclusive agreement with Landmark Graphics Corporation (Halliburton). 



You:  which companies do lmkr collaborate with?


...


Assistant: LMKR specializes in providing geoscience and petroleum-related IT/E&P services, reservoir-centric interpretation, modeling and analytics software, mobile technology solutions, E&P data services, smart urbanization, agri-tech, big data services, AI-driven technology solutions, and consulting. 



You:  what was the gverse geographics university grant?


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



Assistant: LMKR's University Grant Program provides universities with access to highly developed, industry leading software like GVERSE GeoGraphix. This software exposure prepares students for real-life situations and job opportunities within the E&P industry. In specific instances, grants valued at approximately US $4.5 million and US $8.2 million were provided to California University of Pennsylvania and West Virginia University, respectively. These grants granted students enrolled in the geology and geosciences programs at these universities access to 75 software licenses each. 



You:  exit


## 10. Gradio Web Application

A full web interface is built with Gradio:

Features:

* Chatbot window

* Real-time conversation

* Persistent message formatting

* Clean separation of user vs assistant roles

* Support for Enter-to-Send

* Public shareable link

In [12]:
# updated gradio app 
import gradio as gr
def lmkr_chat(user_message, history_messages):
    """
    Gradio chat handler.

    user_message: str
    history_messages: list[{"role": "...", "content": "..."}]
    """
    if history_messages is None:
        history_messages = []

    # Build (user, assistant) pairs for the LMKR prompt
    pairs = []
    last_user = None
    for m in history_messages:
        if m["role"] == "user":
            last_user = m["content"]
        elif m["role"] == "assistant" and last_user is not None:
            pairs.append((last_user, m["content"]))
            last_user = None

    chat_history_str = history_to_str(pairs)
    answer = lmkr_answer(user_message, chat_history_str)

    # Append new messages in messages-format for Gradio
    history_messages = history_messages + [
        {"role": "user", "content": user_message},
        {"role": "assistant", "content": answer},
    ]

    return history_messages, history_messages


def handle_user(user_message, history_messages):
    """
    Wrapper so we can also clear the textbox.
    """
    history, state_out = lmkr_chat(user_message, history_messages)
    return "", history, state_out


with gr.Blocks() as demo:
    gr.Markdown("# 💬 LMKR Assistant\nAsk anything about LMKR (company, products, services, contact info, etc.).")

    chatbot = gr.Chatbot(height=500)
    state = gr.State([])

    with gr.Row():
        msg = gr.Textbox(
            label="Your question",
            placeholder="e.g. What is LMKR? or What is GVERSE?",
            scale=4,
        )
        send_btn = gr.Button("Send", variant="primary", scale=1)

    send_btn.click(
        handle_user,
        inputs=[msg, state],
        outputs=[msg, chatbot, state],
    )

    msg.submit(
        handle_user,
        inputs=[msg, state],
        outputs=[msg, chatbot, state],
    )

demo.launch()


* Running on local URL:  http://127.0.0.1:7860
It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

* Running on public URL: https://8de6b3f687dddd68bf.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


## 11. Results and Evaluation
✔ Strengths

* Accurate LMKR-specific responses

* Fast semantic retrieval

* Efficient model inference due to 4-bit quantization

* Clean, professional answers through prompt engineering


❗ Limitations

* Only .txt files are ingested

* Multi-modal (PDF/image) not yet supported

* Retrieval limited to top-k without metadata filtering

* No long-term memory or session storage